# Assistants File Search
본 노트북에서는, [Azure OpenAI Assistants (preview)의 File Search 툴](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/file-search?tabs=python)을 이용하여 Azure OpenAI 모델 외부의 지식 (제품 정보, 사용자 제공 문서등)으로 Assistant를 보강하는 방법을 보여줍니다. OpenAI는 문서를 자동으로 구문 분석 및 청킹하고, 임베딩을 생성 및 저장하고, 벡터 및 키워드 검색을 모두 사용하여 관련 콘텐츠를 검색하여 사용자 쿼리에 답변합니다.

Azure OpenAI 리소스가 사전에 생성되어 있어야 합니다.

### 소요 시간

이 노트북을 실행하는데는 15분 정도 소요됩니다.

In [ ]:
# Install the packages\
%pip install requests openai~=1.30.5

### 파라미터
Azure OpenAI의 리소스에서 아래의 설정 정보를 복사하여 업데이트 하세요.

In [2]:
import os
import json
import requests
import time
from dotenv import load_dotenv
from openai import AzureOpenAI

In [3]:
load_dotenv()
azure_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OAI_KEY")
deployment_name = os.getenv("AZURE_OAI_DEPLOYMENT")
api_version = "2024-02-15-preview"

### Enable File Search

In [4]:
client = AzureOpenAI(
        api_key = aoai_api_key,  
        api_version="2024-05-01-preview",
        azure_endpoint = azure_endpoint
        )

assistant = client.beta.assistants.create(
    name="Document Analyst Assistant",
    instructions="You are an expert role library analyst. Use your knowledge base to answer questions about roles.",
    model=deployment_name,  # Correct model name
    tools=[{"type": "file_search"}],
)

### File Search를 위한 File Uplaod
파일에 액세스 하기 위해 File Search 툴은 Vector Store 객체를 사용합니다. 파일을 업로드하고 파일을 포함할 Vector Store를 만듭니다. Vector Store가 만들어지면 모든 컨텐츠 처리가 완료 되었는지 확인하기 위해 모든 파일이 "in_progress" 상태를 벗어날때까지 Polling 해야합니다. SDK는 업로드 및 Polling을 위한 도우미를 제공합니다.

In [ ]:
client = AzureOpenAI(
  api_key = aoai_api_key,  
  api_version="2024-05-01-preview",
  azure_endpoint = azure_endpoint
)

# Create a vector store called "Role Library"
vector_store = client.beta.vector_stores.create(name="Role Library")
 
# Ready the files for upload to OpenAI
file_paths = ["file_directory/role_library.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

### Assistant가 새로운 Vector Store를 사용하도록 업데이트
업로드한 파일이 액세스될 수 있도록, Assistant의 "tool_resources"를 새로운 "vector_store" ID로 업데이트 합니다.

In [6]:
#Update the assistant to use the new vector store
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

### Thread 생성
스레드에 메시지 첨부 파일로 파일을 첨부할 수도 있습니다. 이렇게 함으로써, 스레드에 연결된 다른 'vector_store"가 생성되거나, 만약 이 스레드에 이미 Vector Store가 붙어있다면, 기존의 스레드에 파일을 첨부할 수도 있습니다. 이 스레드의 "Run"을 생성할 때, File Search 툴은 당신의 Assistant의 "vector_store"와 이 Thread의 "vector_store"에서 모두 쿼리 할 것입니다.

In [ ]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("file_directory/role_library_01.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Summarize the CTO roles in the library.",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

Vector Store는 마지막 활성 상태 (Vector Store가 실행의 일부였던 마지막 시간으로 정의 됨)로부터 7일이라는 기본 만료 정책이 있는 메시지 첨부 파일을 사용하여 만들어집니다. 이 기본값은 벡터 스토리지 비용을 관리하는데 도움이 됩니다. 이러한 만료 정책은 언제든지 재정의 할 수 있습니다.

### Run을 생성하고 결과를 확인
Run을 생성하고, 모델이 File Search 툴을 이용하여 사용자 질의에 대한 응답을 제공하는지 관찰합니다.

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please help to find out a job and role definition.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()